In [ ]:
import os
import zipfile
import sqlite3
from reportlab.pdfgen import canvas

def generate_pdf(file_path, tracking_url):
    """
    Generate a PDF with an embedded remote image for tracking.
    The image is drawn at an invisible 1x1 pixel size.
    """
    c = canvas.Canvas(file_path)
    # Draw an image from the tracking URL (assumes the PDF viewer loads remote images)
    c.drawImage(tracking_url, 0, 0, width=1, height=1, mask='auto')
    c.showPage()
    c.save()
    print(f"PDF honeytoken generated: {file_path}")

def generate_sql(file_path, tracking_url):
    """
    Generate a SQL file with a comment that includes the tracking URL.
    Note: This does not auto-trigger an HTTP request.
    """
    with open(file_path, "w") as f:
        f.write("-- Honeytoken SQL file\n")
        f.write(f"-- Remote Tracker: {tracking_url}\n")
        f.write("SELECT 'Honeytoken';\n")
    print(f"SQL honeytoken generated: {file_path}")

def generate_zip(file_path, tracking_url):
    """
    Generate a ZIP file that contains an HTML file.
    The HTML file embeds a remote image, so if the HTML is opened in a browser,
    the image request can be logged.
    """
    html_content = f"""<html>
  <body>
    <img src="{tracking_url}" width="1" height="1" alt="tracker" />
    <p>This is a honeytoken contained within a zip file.</p>
  </body>
</html>"""
    # Create a temporary HTML file
    tmp_html = "tracker.html"
    with open(tmp_html, "w") as f:
        f.write(html_content)
    
    # Package the HTML file in a ZIP archive
    with zipfile.ZipFile(file_path, "w") as zipf:
        zipf.write(tmp_html)
    os.remove(tmp_html)
    print(f"ZIP honeytoken generated: {file_path}")

def generate_db(file_path, tracking_url):
    """
    Generate a dummy SQLite database file.
    We create a table and insert a row that includes the tracking URL.
    While opening the DB won’t automatically trigger a remote request,
    the URL will be embedded as a traceable record.
    """
    conn = sqlite3.connect(file_path)
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS honeytoken (id INTEGER PRIMARY KEY, info TEXT)")
    c.execute("INSERT INTO honeytoken (info) VALUES (?)", (f"Remote Tracker: {tracking_url}",))
    conn.commit()
    conn.close()
    print(f"DB honeytoken generated: {file_path}")

def generate_bak(file_path, tracking_url):
    """
    Generate a backup file (BAK) that contains a hidden tracking URL in plain text.
    """
    with open(file_path, "w") as f:
        f.write("This is a honeytoken backup file.\n")
        f.write(f"Remote Tracker: {tracking_url}\n")
    print(f"BAK honeytoken generated: {file_path}")

def generate_honeytoken_file(file_name, tracking_url):
    """
    Dispatch to the correct generator based on file extension.
    """
    ext = os.path.splitext(file_name)[1].lower()
    if ext == ".pdf":
        generate_pdf(file_name, tracking_url)
    elif ext == ".sql":
        generate_sql(file_name, tracking_url)
    elif ext == ".zip":
        generate_zip(file_name, tracking_url)
    elif ext == ".db":
        generate_db(file_name, tracking_url)
    elif ext == ".bak":
        generate_bak(file_name, tracking_url)
    else:
        # For any other file type, write a simple text file with the tracking URL.
        with open(file_name, "w") as f:
            f.write(f"Remote Tracker: {tracking_url}\n")
        print(f"Generic honeytoken generated: {file_name}")


In [ ]:
# Define your unique tracking URL, e.g., pointing to your server’s tracking endpoint.
tracking_url = "https://yourserver.com/tracker?token=unique_honeytoken_id"

# List of bait file names you want to generate.
filenames = ["bait_document.pdf", "sensitive_export.sql", "backup.zip", "internal.db", "config.bak"]

for fn in filenames:
    generate_honeytoken_file(fn, tracking_url)

In [ ]:
import re
from io import BytesIO
from pathlib import Path

STREAM_OFFSET = 100  # Example offset; you must determine this based on your template

def substitute_stream(header: bytes, stream: bytes, replace: bytes):
    # This helper function would locate the placeholder within the stream
    # and substitute it with your unique token (e.g., a hostname).
    # (Implementation details depend on your PDF template.)
    new_stream = stream.replace(b"Top", replace)
    return header, new_stream

def make_custom_pdf(hostname: bytes, 
                    template: Path, 
                    stream_offset: int = STREAM_OFFSET):
    
    # Read the template PDF as binary
    with open(template, "rb") as fp:
        contents = fp.read()
    
    # Extract stream length using regex on a slice from the stream offset
    stream_size_match = re.match(rb".*\/Length ([0-9]+)\/.*", contents[stream_offset:])
    if not stream_size_match:
        raise ValueError("Could not find stream length")
    stream_size = int(stream_size_match.group(1))
    
    # Find where the stream starts (after 'stream\r\n')
    stream_start = stream_offset + contents[stream_offset:].index(b"stream\r\n") + 8
    stream_header = contents[stream_offset:stream_start]
    stream = contents[stream_start: stream_start + stream_size]
    
    # Substitute the placeholder with your hostname
    new_header, new_stream = substitute_stream(header=stream_header, stream=stream, replace=hostname)
    
    # Reassemble the PDF contents with the modified stream
    output = BytesIO()
    output.write(contents[0:stream_offset])
    output.write(new_header)
    output.write(new_stream)
    output.write(contents[stream_start + stream_size:])
    new_contents = output.getvalue()
    output.close()
    
    return new_contents

# Example usage:
template_path = Path("pdf_template.pdf")

# Use a unique hostname such as b'unique1234.honeytoken.yourdomain.com'
unique_hostname = b"https://.smartgadgetstore.live/robots.txt"
modified_pdf = make_custom_pdf(unique_hostname, template_path)
with open("honeytoken.pdf", "wb") as f:
    f.write(modified_pdf)
print("Honeytoken PDF created.")


In [ ]:
import random
import re
import zlib
from io import BytesIO
from pathlib import Path

STREAM_OFFSET = 100  # Adjust this based on your PDF template

def _substitute_stream(
    header,
    stream: bytes,
    replace: bytes,
    search: bytes = b"abcdefghijklmnopqrstuvwxyz.zyxwvutsrqponmlkjihgfedcba.aceegikmoqsuwy.bdfhjlnprtvxz",
):
    """
    Replace a placeholder in the PDF stream with the tracking URL.
    Ensures the modified stream is the same size as the original.
    """
    MAX_ATTEMPTS = 100
    old_len = len(stream)
    attempts = 0

    while attempts < MAX_ATTEMPTS:
        candidate_stream = zlib.compress(
            zlib.decompress(stream).replace(search, replace)
        )
        count = 1
        while len(candidate_stream) < old_len and count < 10000:
            padding = "".join(
                [chr(random.randrange(65, 90)) for _ in range(count)]
            ).encode()
            candidate_stream = zlib.compress(
                zlib.decompress(stream).replace(search, replace + b"/" + padding)
            )
            count += 1
        if len(candidate_stream) == old_len:
            break
        attempts += 1

    if attempts == MAX_ATTEMPTS:
        raise Exception(
            f"Failed to adjust stream size after {MAX_ATTEMPTS} attempts."
        )

    return header, candidate_stream


def make_canary_pdf(
    hostname: bytes, template: Path, stream_offset: int = STREAM_OFFSET
):
    """
    Create a PDF with an embedded tracking URL.
    """
    with open(template, "rb") as fp:
        contents = fp.read()

    # Dynamically locate the /Length property in the PDF content
    length_match = re.search(rb"/Length (\d+)", contents)
    if not length_match:
        raise ValueError("Could not find stream length in the PDF template.")
    stream_size = int(length_match.group(1))

    stream_start = stream_offset + contents[stream_offset:].index(b"stream\r\n") + 8
    stream_header = contents[stream_offset:stream_start]
    # Extract the stream data up to the 'endstream' marker to ensure completeness
    stream_end = stream_start + stream_size
    if b"endstream" in contents[stream_start:]:
        stream_end = stream_start + contents[stream_start:].index(b"endstream")
    stream = contents[stream_start:stream_end]
    
    print(f"Stream size: {stream_size}, Stream start: {stream_start}, Stream end: {stream_end}")
    # Replace the placeholder in the stream with the tracking URL
    stream_header, stream = _substitute_stream(
        header=stream_header, stream=stream, replace=hostname
    )

    # Reassemble the PDF with the modified stream
    output = BytesIO()
    output.write(contents[:stream_offset])
    output.write(stream_header)
    output.write(stream)
    output.write(contents[stream_start + stream_size:])
    new_contents = output.getvalue()
    output.close()

    return new_contents




In [ ]:
import random
import re
import zlib
from io import BytesIO
from pathlib import Path

#from canarytokens.constants import CANARY_PDF_TEMPLATE_OFFSET as STREAM_OFFSET

# PDF_FILE=settings.CANARY_PDF_TEMPLATE
# STREAM_OFFSET=settings.CANARY_PDF_TEMPLATE_OFFSET
STREAM_OFFSET = 793
# CANARY_PDF_TEMPLATE_OFFSET=793


def _substitute_stream(
    header,
    stream: bytes,
    replace: bytes,
    search: bytes = b"abcdefghijklmnopqrstuvwxyz.zyxwvutsrqponmlkjihgfedcba.aceegikmoqsuwy.bdfhjlnprtvxz",
):
    # Ohhhh, this is nasty. Instead of trying to get the xref positions right,
    # we're going to brute-force a URL that's the right size after compression.
    # Give up after 100 attempts.

    MAX_ATTEMPTS = 100

    old_len = len(stream)
    attempts = 0
    while attempts < MAX_ATTEMPTS:
        candidate_stream = zlib.compress(
            zlib.decompress(stream).replace(search, replace)
        )
        count = 1
        while len(candidate_stream) < old_len and count < 10000:
            padding = "".join(
                [chr(random.randrange(65, 90)) for x in range(0, count)]
            ).encode()
            candidate_stream = zlib.compress(
                zlib.decompress(stream).replace(search, replace + b"/" + padding)
            )
            count += 1
        if old_len == len(candidate_stream):
            break
        attempts += 1

    if attempts == MAX_ATTEMPTS:
        raise Exception(
            "Dammit, new PDF is too big after {attempts} attempts, ({new_len} > {old_len})".format(
                attempts=attempts, new_len=len(candidate_stream), old_len=old_len
            )
        )

    return (header, candidate_stream)


def make_canary_pdf(
    hostname: bytes, template: Path, stream_offset: int = STREAM_OFFSET
):
    with open(template, "rb") as fp:
        contents = fp.read()

    stream_size = int(
        re.match(rb".*\/Length ([0-9]+)\/.*", contents[stream_offset:]).group(1)
    )
    stream_start = stream_offset + contents[stream_offset:].index(b"stream\r\n") + 8
    stream_header = contents[stream_offset:stream_start]
    stream = contents[stream_start : stream_start + stream_size]  # noqa: E203

    (stream_header, stream) = _substitute_stream(
        header=stream_header, stream=stream, replace=hostname
    )

    output = BytesIO()
    output.write(contents[0:stream_offset])
    output.write(stream_header)
    output.write(stream)
    output.write(contents[stream_start + stream_size :])  # noqa: E203
    new_contents = output.getvalue()
    output.close()

    return new_contents

# Example usage
template_path = Path("/Users/nicolaiveiglinarends/dtu-honeypot-thesis/snare/snare/tests/pdf_template_token.pdf")
output_path = Path("/Users/nicolaiveiglinarends/dtu-honeypot-thesis/snare/snare/tests/honeytoken.pdf")
tracking_url = b"https://smartgadgetstore.live/pdf_template.pdf"

# Generate the modified PDF
modified_pdf = make_canary_pdf(tracking_url, template_path)
with open(output_path, "wb") as f:
    f.write(modified_pdf)

print(f"Honeytoken PDF created: {output_path}")

In [ ]:
import re
import zlib
from io import BytesIO
from pathlib import Path

# Set the offset where the target content stream begins in your template.
STREAM_OFFSET = 793  # Adjust based on your PDF template

def substitute_stream(header: bytes, stream: bytes, replacement: bytes,
                      placeholder: bytes = b"PLACEHOLDER_TOKEN") -> (bytes, bytes):
    """
    Decompress the given PDF stream, replace a placeholder with the replacement,
    and then recompress while attempting to match the original stream length.

    If the new compressed stream is shorter than the original, we pad the decompressed
    data (with spaces) until the recompressed length equals the original.
    """
    original_length = len(stream)
    # Decompress the original stream data
    try:
        decompressed = zlib.decompress(stream)
    except zlib.error as e:
        raise Exception("Error decompressing PDF stream") from e

    # Replace the placeholder in the decompressed data
    new_decompressed = decompressed.replace(placeholder, replacement)

    # If the replacement makes the decompressed data shorter, pad it with spaces.
    if len(new_decompressed) < len(decompressed):
        diff = len(decompressed) - len(new_decompressed)
        new_decompressed += b' ' * diff  # pad with spaces

    # Recompress the new decompressed data
    new_stream = zlib.compress(new_decompressed)

    # Adjust padding if the compressed stream length doesn't match original.
    attempts = 0
    while len(new_stream) != original_length and attempts < 100:
        if len(new_stream) < original_length:
            # If too short, add one more space to the decompressed data.
            new_decompressed += b' '
        else:
            # If too long, try removing trailing spaces.
            new_decompressed = new_decompressed.rstrip(b' ')
        new_stream = zlib.compress(new_decompressed)
        attempts += 1

    if len(new_stream) != original_length:
        raise Exception(f"Could not adjust stream length after {attempts} attempts "
                        f"({len(new_stream)} != {original_length})")

    return header, new_stream

def make_pdf_honeytoken(template: Path, output: Path, replacement: bytes) -> None:
    """
    Creates a honeytoken PDF by reading a template PDF, replacing a placeholder in a
    content stream with a tracking token (replacement), and writing the modified PDF.
    """
    # Read the entire template PDF into memory
    with open(template, "rb") as fp:
        contents = fp.read()

    # Extract the stream length from the PDF using a regex.
    # This example looks for a pattern like "/Length 1234" starting from STREAM_OFFSET.
    match = re.search(rb".*?/Length\s+([0-9]+)", contents[STREAM_OFFSET:])
    if not match:
        raise Exception("Could not find stream length in template")
    stream_length = int(match.group(1))

    # Find where the stream data starts.
    # We assume the stream starts after "stream\r\n".
    stream_marker = b"stream\r\n"
    try:
        stream_start = STREAM_OFFSET + contents[STREAM_OFFSET:].index(stream_marker) + len(stream_marker)
    except ValueError:
        raise Exception("Could not find stream marker in template")

    # Split the PDF into three parts:
    # 1. Before the stream (from start of file to STREAM_OFFSET)
    # 2. The stream itself (from stream_start to stream_start + stream_length)
    # 3. The remainder of the file.
    header = contents[STREAM_OFFSET:stream_start]
    stream_data = contents[stream_start:stream_start + stream_length]
    footer = contents[stream_start + stream_length:]

    # Substitute the placeholder with your honeytoken in the stream data.
    new_header, new_stream = substitute_stream(header, stream_data, replacement)

    # Reassemble the PDF file.
    new_contents = contents[:STREAM_OFFSET] + new_header + new_stream + footer

    # Write the modified PDF to output.
    with open(output, "wb") as out_file:
        out_file.write(new_contents)
 
    print(f"Honeytoken PDF created: {output}")

# Example usage:
template_path = Path("/Users/nicolaiveiglinarends/dtu-honeypot-thesis/snare/snare/tests/pdf_template_honeytoken.pdf")  # Your PDF template file containing PLACEHOLDER_TOKEN
output_path = Path("/Users/nicolaiveiglinarends/dtu-honeypot-thesis/snare/snare/tests/honeytoken.pdf")
# Example replacement: a tracking URL or unique token.
# Ensure the replacement is of appropriate length or allow the padding loop to adjust.
replacement_token = b"https://webhook.site/cb2663e7-634a-411b-b21d-3eadd61c53ae"  

make_pdf_honeytoken(template_path, output_path, replacement_token)


In [ ]:
#!/usr/bin/env python3

import requests, sys, json
from typing import Optional, Dict

TOKENS_URL='https://canarytokens.org'

def _gen_req_data(type: str, email: str, memo: str) -> Dict[str, str]:
    return {
        'type': type,
        'email': email,
        'memo': memo,
        'fmt': '',
        'webhook': '',
        'redirect_url': '',
        'cmd_process': '',
        'azure_id_cert_file_name': '',
        'clonedsite': '',
        'sql_server_table_name': 'TABLE1',
        'sql_server_view_name': 'VIEW1',
        'sql_server_function_name': 'FUNCTION1',
        'sql_server_trigger_name': 'TRIGGER1'
    }

def get_pdf_token(email : str, memo : str) -> Optional[str]:
    '''
    Returns a web bug token URL given an email and memo
    '''
    req_data = _gen_req_data('pdf', email, memo)
    res = requests.post(TOKENS_URL + '/generate', data=req_data)
    if res.status_code != 200:
        print(f"Error: {res.status_code} - {res.text}", file=sys.stderr)
        return None
    
    return res.json().get('token_url', None)

def get_aws_token(email : str, memo: str) -> Optional[Dict[str, str]]:
    '''
    Returns a dict of an AWS access key given an email and memo
    '''
    req_data = _gen_req_data('aws_keys', email, memo)
    res = requests.post(TOKENS_URL + '/generate', data=req_data)
    if res.status_code != 200:
        return None
    return {'access_key': res.json().get('aws_access_key_id'), 'secret_key': res.json().get('aws_secret_access_key')}


In [ ]:
response = get_pdf_token("nicolaiveiglin@gmail.com", "Test")

In [ ]:
response

In [ ]:
import requests
import json


TOKENS_URL='https://canarytokens.org'
TOKENS_DOWNLOAD_URL = 'https://canarytokens.org/d3aece8093b71007b5ccfedad91ebb11/download'

def generate_token(type: str, memo : str, webhook: str = ''):
    '''
    Returns a web bug token URL given an email and memo
    '''
    req_data = {
        'type': type,
        'memo': memo,
        'webhook_url': webhook
    }
    res = requests.post(TOKENS_URL + '/generate', data=req_data)
    if res.status_code != 200:
        print(f"Error: {res.status_code} - {res.text}")
        return None
    return res.json()

def _downloaded_token_file(self, type: str, auth: str, token: str):

    # Map the file type to the correct fmt
    file_extensions = {
        'adobe_pdf': 'pdf',
        'ms_word': 'msword',
        'ms_excel': 'msexcel',
    }
    fmt = file_extensions.get(type)
    if not fmt:
        print(f"Unsupported file type: {type}", "ERROR")
        return None
    
    # Define the download URL parameters
    params = {
        'fmt': fmt,
        'auth': auth,
        'token': token
    }
    # Make the GET request to download the file
    response = requests.get(TOKENS_DOWNLOAD_URL, params=params, allow_redirects=True)
    # Check if the request was successful
    if response.status_code == 200:
        print(f"File content successfully downloaded", "INFO")
        return response.content
    else:
        print(f"Failed to download content: {response.status_code} - {response.text}", "ERROR")

In [ ]:
data = generate_token("adobe_pdf", "PDF - Triggered", webhook="http://localhost:5003/webhook")

In [ ]:
data

In [ ]:
%pip install ctransformers

In [ ]:
from huggingface_hub import snapshot_download

# Download model files to cache
snapshot_download(repo_id="TheBloke/zephyr-7B-beta-GGUF", repo_type="model", allow_patterns='*Q4_0.gguf')


In [6]:
from ctransformers import AutoModelForCausalLM

# Specify the repo, the exact GGUF filename, and the underlying model type
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-beta-GGUF",
    model_file="zephyr-7b-beta.Q4_0.gguf",
    model_type="mistral",
    local_files_only=True            # or N layers to offload to GPU
)

In [5]:
g = llm.generate("Hello, how are you?")

In [7]:
llm

In [8]:
prompt = "What is the capital of France?"
# Generate a response
response = llm(prompt, max_new_tokens=50)
print(response)

KeyboardInterrupt: 

In [ ]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="TheBloke/zephyr-7B-beta-GGUF",
    tokenizer="TheBloke/zephyr-7B-beta-GGUF",
    local_files_only=True
)

In [ ]:
output = generator(
    "In a distant future, AI will",
    max_length=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)
for seq in output:
    print(seq["generated_text"])


In [ ]:
from ollama import chat

response = chat(
    model='mistral',
    messages=[
      {'role': 'system',  'content': 'You are a helpful assistant.'},
      {'role': 'user',    'content': 'What is the capital of France?'}
    ]
)
print(response['message']['content'])

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [1]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF",
	filename="Mistral-7B-Instruct-v0.3.IQ1_M.gguf",
)
llama_response = llm.create_chat_completion(
    messages = [
    	{
    		"role": "user",
    		"content": "What is the capital of France?"
    	}
    ]
)

Mistral-7B-Instruct-v0.3.IQ1_M.gguf:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

llama_model_load_from_file_impl: using device Metal (Apple M1) - 5461 MiB free
llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from /Users/nicolaiveiglinarends/.cache/huggingface/hub/models--MaziyarPanahi--Mistral-7B-Instruct-v0.3-GGUF/snapshots/ce89f595755a4bf2e2e05d155cc43cb847c78978/./Mistral-7B-Instruct-v0.3.IQ1_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mistral-7B-Instruc...
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096


In [2]:
llama_response = llm.create_chat_completion(
    messages = [
    	{
    		"role": "user",
    		"content": "What is the capital of France?"
    	}
    ]
)

Llama.generate: 11 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    5593.04 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =  104065.95 ms /    90 runs   ( 1156.29 ms per token,     0.86 tokens per second)
llama_perf_context_print:       total time =  104103.13 ms /    91 tokens


In [4]:
llama_response['choices'][0]['message']['content']

" Paris is the capital of France. However, if you're referring to the capital of France in terms of the country's government, that would be Brussels, as the capital of the European Union is located in Brussels, Belgium. But if you're referring to the capital city of France, Paris is the largest city in France and is the center of French politics, as it is the seat of the French government."